In [1]:
import time
from datetime import datetime
from pathlib import Path

import pandas as pd
import requests


In [2]:
# --- Configuration ---
BASE_URL = "https://api.company-information.service.gov.uk/advanced-search/companies"
API_KEY = "b8ca8790-2ffe-453e-a4e5-8e45b935e19a"  # same key as other scripts
OUTPUT_DIR = Path("/home/ae25872/codebase/proai/Common-Crawl---Autumn-2025")
TARGET_COUNT = 500  # number of companies to fetch per SIC code
PAGE_SIZE = 100
RATE_LIMIT_DELAY = 1.0  # seconds between API calls
MAX_RETRIES = 3

# Update this list with the SIC codes you want to fetch
sic_codes = [
    "01270",
    "01210",
    "01220",
    "64410",
    "64191", 
    "65110", # example placeholder, replace later
]

session = requests.Session()
session.auth = (API_KEY, "")
session.headers.update({"Accept": "application/json"})


In [3]:
def fetch_companies_batch(sic_code: str, start_index: int, size: int = PAGE_SIZE) -> dict | None:
    params = {
        "company_status": "active",
        "sic_codes": sic_code,
        "size": size,
        "start_index": start_index,
    }

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = session.get(BASE_URL, params=params, timeout=30)

            if resp.status_code == 429:
                wait = int(resp.headers.get("Retry-After", RATE_LIMIT_DELAY * attempt))
                print(f"  ⚠️ Rate limited. Waiting {wait}s before retry {attempt}/{MAX_RETRIES}...")
                time.sleep(wait)
                continue

            resp.raise_for_status()
            return resp.json()
        except requests.HTTPError as err:
            print(f"  ✗ HTTP error {resp.status_code}: {resp.text[:200]}")
            if resp.status_code >= 500 and attempt < MAX_RETRIES:
                time.sleep(RATE_LIMIT_DELAY * attempt)
            else:
                return None
        except requests.RequestException as err:
            print(f"  ✗ Request error: {err}")
            if attempt < MAX_RETRIES:
                time.sleep(RATE_LIMIT_DELAY * attempt)
            else:
                return None

    return None


def collect_companies_for_sic(sic_code: str, target_count: int = TARGET_COUNT) -> list[dict]:
    collected: list[dict] = []
    start_index = 0

    while len(collected) < target_count:
        batch = fetch_companies_batch(sic_code, start_index)
        if not batch or not batch.get("items"):
            break

        collected.extend(batch["items"])

        received = len(batch["items"])
        start_index += received

        if received < PAGE_SIZE:
            break

        time.sleep(RATE_LIMIT_DELAY)

    return collected[:target_count]


def normalize_companies(items: list[dict]) -> pd.DataFrame:
    if not items:
        return pd.DataFrame()
    df = pd.json_normalize(items, sep=".")
    return df


In [4]:
all_frames: list[pd.DataFrame] = []
output_files: dict[str, Path] = {}

for sic in sic_codes:
    print(f"\n=== Fetching companies for SIC {sic} ===")
    items = collect_companies_for_sic(sic)
    print(f"  Retrieved {len(items)} companies")

    df_sic = normalize_companies(items)
    if df_sic.empty:
        print("  No results for this code; skipping save")
        continue

    df_sic.insert(0, "input_sic_code", sic)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = OUTPUT_DIR / f"sic_{sic}_companies_{timestamp}.csv"
    df_sic.to_csv(output_path, index=False)
    output_files[sic] = output_path

    # Also save with only the SIC code in the name
    output_path_simple = OUTPUT_DIR / f"{sic}.csv"
    df_sic.to_csv(output_path_simple, index=False)

    print(f"  Saved {len(df_sic)} rows to {output_path} and {output_path_simple}")
    all_frames.append(df_sic)

if all_frames:
    combined_df = pd.concat(all_frames, ignore_index=True)
    display(combined_df.head())
    print(f"\nCombined total rows: {len(combined_df)}")
else:
    combined_df = pd.DataFrame()
    print("No data collected. Check SIC codes or API credentials.")



=== Fetching companies for SIC 01270 ===
  Retrieved 73 companies
  Saved 73 rows to /home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/sic_01270_companies_20251121_122357.csv and /home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/01270.csv

=== Fetching companies for SIC 01210 ===
  Retrieved 276 companies
  Saved 276 rows to /home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/sic_01210_companies_20251121_122359.csv and /home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/01210.csv

=== Fetching companies for SIC 01220 ===
  Retrieved 79 companies
  Saved 79 rows to /home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/sic_01220_companies_20251121_122359.csv and /home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/01220.csv

=== Fetching companies for SIC 64410 ===
  ✗ HTTP error 404: 
  Retrieved 0 companies
  No results for this code; skipping save

=== Fetching companies for SIC 64191 ===
  Retrieved 500 companies
  Saved 500 rows to /home/ae25872/codebase/pr

,input_sic_code,company_name,company_number,company_status,company_type,kind,date_of_creation,sic_codes,links.company_profile,registered_office_address.address_line_1,registered_office_address.address_line_2,registered_office_address.locality,registered_office_address.postal_code,registered_office_address.region,registered_office_address.country,company_subtype
0,01270,CAMHAYLE CORNISH HOPS LIMITED,10191081,active,ltd,search-results#company,2016-05-20,[01270],/company/10191081,Trezerricks,Edmonton,Wadebridge,PL27 7JA,Cornwall,United Kingdom,NaN
1,01270,WHC LAB LIMITED,NI678301,active,ltd,search-results#company,2021-04-12,[01270],/company/NI678301,18 Mopack Business Park Ballycolman Road,Strabane,County Tyrone,BT82 9PH,NaN,Northern Ireland,NaN
2,01270,HEBRIDEAN TEA COMPANY LIMITED,SC598578,active,ltd,search-results#company,2018-05-30,[01270],/company/SC598578,63 Kenneth Street,Stornoway,Isle Of Lewis,HS1 2DS,NaN,Scotland,NaN
3,01270,ORCHARD RISE AGRICULTURE LIMITED,10571223,active,ltd,search-results#company,2017-01-18,[01270],/company/10571223,Lefevres Limited 24 Bell Lane,Blackwater,Camberley,GU17 0NW,Surrey,England,NaN
4,01270,SIBLING WINERY LIMITED,11577841,active,ltd,search-results#company,2018-09-19,"[01270, 11020, 11030]",/company/11577841,3 Hill Top,Loxley,Warwick,CV35 9JU,Warwickshire,United Kingdom,NaN



Combined total rows: 1428
